<a href="https://colab.research.google.com/github/dg4271/OpenQA-korean/blob/main/openqa_using_haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Haystack을 활용한 한국어 Open-domain 질의 응답
> Open-domain Question Answering in Korean using Haystack <br>
> KorQuad 데이터셋으로 finetuning 된 KoELECTRA 모델 사용 <br>
> Haystack의 document retriever를 이용해 context 없이 question만 입력하여 답을 찾을 수 있도록 함 <br>
> github.com/dg4271

## 라이브러리 설치
* haystack: Open-doamin Question Answering framework
* pytorch

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack.git
#!pip install urllib3==1.25.4
!pip install torch==1.6.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

### Import Library

In [20]:
from haystack import Finder
# from haystack.preprocessor.cleaning import clean_wiki_text
# from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
# from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

12/31/2020 06:10:51 - INFO - faiss -   Loading faiss with AVX2 support.
12/31/2020 06:10:51 - INFO - faiss -   Loading faiss.


## 문서 데이터
* korquad 1.0 데이터의 context를 색인할 문서로 사용
* 이미 어느정도 전처리/정제 되어 있음

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [45]:
import json
import pprint

train_path = '/gdrive/My Drive/dataset/KorQuAD_v1.0_train.json'
dev_path = '/gdrive/My Drive/dataset/KorQuAD_v1.0_dev.json'

# load korquad data
dev_json_dic = {}
with open(train_path, 'r') as f:
  dev_json_dic = json.load(f)
pprint.pprint(dev_json_dic['data'][10:15])

# parsing data
dicts = []
for doc in dev_json_dic['data']:
  for context_pair in doc['paragraphs']:
    text = " ".join(context_pair['context'].split('\n'))
    dicts.append({"name": doc['title'], "text": text})

# pprint.pprint(dicts[:5])

1420
dict_keys(['paragraphs', 'title'])
[{'paragraphs': [{'context': '하지만 2006~2007 시즌 이후에도 왼쪽 무릎 연골 파열로 인한 부상으로 수술을 '
                             '받아야 했다. 재활을 마치자마자 2007년 배구 월드컵 국가대표로 선발되어 거의 모든 '
                             '경기를 뛰었으며, 월드컵 일정을 마치고 난 뒤에는 소속 팀으로 돌아와 4달 간의 '
                             '2007~2008 시즌을 소화해야 했다. 2007~2008 시즌에서 소속 팀 흥국생명은 '
                             '챔피언 결정전에서 인천 GS칼텍스에게 덜미를 잡혀 통합 우승에는 실패했지만, 김연경은 '
                             '역대 최고의 공격 성공률인 47.59%을 기록하면서 공격상을 3년 연속 수상했고, '
                             '흥국생명을 정규리그 1위로 올리는 데에 성공하며 정규리그 MVP도 3년 연속 수상했다. '
                             '그러나 두 번째 수술 후 무리한 일정 탓에 베이징 올림픽 최종 예선을 앞두고 무릎 '
                             '연골이 다시 파열되며 3년 연속으로 수술대에 오르고 말았다.',
                  'qas': [{'answers': [{'answer_start': 232, 'text': '김연경'}],
                           'id': '6488392-0-0',
                           'question': '07~08 시즌 당시 흥국생명 소속으로 공격상과 정규리그 MVP 를 '
                                       '3년연속  수상한 사람은?

#Document Store
간단하게 In memory에서의 DocumnetStore를 사용해볼 수 있다.

{"name": "<some-document-name>, "text": "<the-actual-text>"}

형태의 dictionary 들을 원소로 갖는 list를 document_store에 색인할 수 있음


In [21]:
# In-Memory Document Store
from haystack.document_store.memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()
document_store.write_documents(dicts)

### Retriever, Reader, & Finder 초기화

Retriever
Retrievers help narrowing down the scope for the Reader to smaller units of text where a given question could be answered.

With InMemoryDocumentStore or SQLDocumentStore, you can use the TfidfRetriever. For more retrievers, please refer to the tutorial-1.

In [29]:
# An in-memory TfidfRetriever based on Pandas dataframes

from haystack.retriever.sparse import TfidfRetriever
# Now, let's write the docs to our DB.
retriever = TfidfRetriever(document_store=document_store)

12/31/2020 06:13:37 - INFO - haystack.retriever.sparse -   Found 9681 candidate paragraphs from 9681 docs in DB


### Reader

A Reader scans the texts returned by retrievers in detail and extracts the k best answers. They are based
on powerful, but slower deep learning models.

Haystack currently supports Readers based on the frameworks FARM and Transformers.
With both you can either load a local model or one from Hugging Face's model hub (https://huggingface.co/models).

**Here:** a medium sized RoBERTa QA model using a Reader based on FARM (https://huggingface.co/deepset/roberta-base-squad2)

**Alternatives (Reader):** TransformersReader (leveraging the `pipeline` of the Transformers package)

**Alternatives (Models):** e.g. "distilbert-base-uncased-distilled-squad" (fast) or "deepset/bert-large-uncased-whole-word-masking-squad2" (good accuracy)

**Hint:** You can adjust the model to return "no answer possible" with the no_ans_boost. Higher values mean the model prefers "no answer possible"

### KoELECTRA
본 구현에서는 공개된 한국어 언어 모델인 [KoELECTRA](https://github.com/monologg/KoELECTRA)를 이용하였음
transformers 이미 학습된 모델을 쉽게 다운받을 수 있고, KoELECTRA의 개발자가 이미 KorQuad 1.0에 파인튜닝 된 모델을 올려두었음
Haystack으로 retireval와 reader를 쉽게 연결해 사용 가능함


In [ ]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering, pipeline

# tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
# model = ElectraForQuestionAnswering.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")
model = ElectraForQuestionAnswering.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")

In [49]:
# Alternative:
reader = TransformersReader(model_name_or_path=model, tokenizer=tokenizer, use_gpu=-1)

In [50]:
finder = Finder(reader, retriever)

12/31/2020 06:34:59 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.


In [51]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
# question = "세계에서 우라늄을 가장 많이 생산하는 나라는?"
# question = "초우라늄 원소가 존재할 가능성을 처음 제안한 사람은?"
# question = "캘리포늄-249의 반감기는 몇 년인가?"
question = "일본이 신라에 사신을 보냈다가 쫓겨나게 된 곳은?"
print_answers(finder.get_answers(question=question, top_k_retriever=1, top_k_reader=5), details="minimal")

12/31/2020 06:35:02 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
12/31/2020 06:35:02 - INFO - haystack.finder -   Got 1 candidates from retriever
12/31/2020 06:35:02 - INFO - haystack.finder -   Reader is looking for detailed answer in 423 chars ...
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


[   {   'answer': '다자이후는',
        'context': '공을 강요하다가 추방당하는 지경에 이르렀다. 신라에서도 사신을 파견했다가 다자이후에서 쫓겨나기도 했다. '
                   '7세기 후반에 설치된 다자이후는 신라와 당나라 등 외국 사신들이 입국할 때 외교 절차를 거치던 곳인데, '
                   '《속일본기》에는 이곳에 온 신라 사신들을 그냥 돌려보'},
    {'answer': None, 'context': None},
    {   'answer': '다자이후는',
        'context': '공을 강요하다가 추방당하는 지경에 이르렀다. 신라에서도 사신을 파견했다가 다자이후에서 쫓겨나기도 했다. '
                   '7세기 후반에 설치된 다자이후는 신라와 당나라 등 외국 사신들이 입국할 때 외교 절차를 거치던 곳인데, '
                   '《속일본기》에는 이곳에 온 신라 사신들을 그냥 돌려보'},
    {   'answer': '다자이후는',
        'context': '공을 강요하다가 추방당하는 지경에 이르렀다. 신라에서도 사신을 파견했다가 다자이후에서 쫓겨나기도 했다. '
                   '7세기 후반에 설치된 다자이후는 신라와 당나라 등 외국 사신들이 입국할 때 외교 절차를 거치던 곳인데, '
                   '《속일본기》에는 이곳에 온 신라 사신들을 그냥 돌려보'},
    {   'answer': '다자이후는',
        'context': '공을 강요하다가 추방당하는 지경에 이르렀다. 신라에서도 사신을 파견했다가 다자이후에서 쫓겨나기도 했다. '
                   '7세기 후반에 설치된 다자이후는 신라와 당나라 등 외국 사신들이 입국할 때 외교 절차를 거치던 곳인데, '
                   '《속일본기》에는 이곳에 온 신라 사신들을 그냥 돌려보'}]
